In [34]:
import pandas as pd
from datetime import datetime
import bcrypt

In [ ]:
df = pd.read_csv('csv/users.csv', encoding='utf-8')
cur = pd.read_csv('out/currency.csv', encoding='utf-8')

In [5]:
df = df.drop(['21.5.2022', 'Profile picture', 'Review1', 'Review2', 'Review3', 'Review4', 'Review5', 'Review6', 'Shutterstock URL', 'Photo ID'], axis = 1)

In [8]:
df.rename(columns = {'PUBLIC':'hasPublicProfile', 'Nationality' : 'nationality', \
                    'Age' : 'age', 'Sex' : 'sex', 'About Me' : 'bio', 'Name' : 'firstName', \
                    'Surname' : 'lastName', 'Username' : 'username'}, inplace = True)

In [35]:
columns = ['id', 'createdDate', 'updatedDate', 'isActive', 'email', 'hash', 'hasPaswordSaved', \
          'phone', 'statusId', 'defaultLocalityId', 'coverImageId', 'level', 'passwordResetHash', 'passwordResetValid', \
          'searchVector', 'searchConfig', 'defaultCurrencyId', 'tempPass']

In [36]:
df = pd.concat([df, pd.DataFrame(columns = columns)])

In [37]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [38]:
def set_public_profile(value):
    if value == 'ANO':
        return True
    return False

In [43]:
def create_pass(value):
    temp = 'Trynego+' + value
    bytePwd = temp.encode('utf-8')
    mySalt = bcrypt.gensalt()
    hash = bcrypt.hashpw(bytePwd, mySalt)
    return hash

In [ ]:
df['id'] = df.index + 1
df['order'] = ''
df['createdDate'] = dt_string
df['updatedDate'] = dt_string
df['hasPublicProfile'] = df['hasPublicProfile'].apply(set_public_profile)
df['age'] = df.age.astype(int)
df['sex'] = df.sex.str.lower()
df['statusId'] = 4
df['level'] = 1
df['hash'] = df.username.apply(create_pass)

In [ ]:
df['isActive'] = True
df['email'] = df.firstName + '.' + df.lastName + '@trynego.com'

In [ ]:
df.head()